In [16]:
#!/usr/bin/env python

from confluent_kafka import Consumer
from message import Message
from schema_registry import SchemaRClient
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry.avro import AvroDeserializer



config = {
        'bootstrap.servers': 'pkc-12576z.us-west2.gcp.confluent.cloud:9092',
        'sasl.username':     '',
        'sasl.password':     '',
        'security.protocol': 'SASL_SSL',
        'sasl.mechanisms':   'PLAIN',
        'group.id':          'kafka-python-getting-started',
        'auto.offset.reset': 'earliest'
    }
consumer = Consumer(config)

In [17]:
def convert_reverse(message,ctx):
    if message is None:
        return None
    return Message(idade=message['idade'],
                name=message['name'],
                team=message['team'])

In [18]:
scr_client = SchemaRClient("https://@psrc-57o65q.us-west2.gcp.confluent.cloud",
                                       "",
                                       ""
                                       )

schema_str = scr_client.get_schema_str(100001)
avro_deserializer = AvroDeserializer(scr_client.client,
                                     schema_str,
                                     convert_reverse)


topic = "topic_0"
consumer.subscribe([topic])


In [19]:


try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:

            pass
        elif msg.error():
            print("ERROR: %s".format(msg.error()))
        else:
            message = avro_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))
            if message is not None:
                print("User record {}: name: {}\n"
                      "\tidade: {}\n"
                      "\tmessage: {}\n"
                      .format(msg.key(), message.name,
                              message.idade,
                              message.team))
except KeyboardInterrupt:
    pass
finally:
    # Leave group and commit final offsets
    consumer.close()

TypeError: Message.__init__() missing 1 required positional argument: 'record_data'

In [ ]:

# Subscribe to topic
topic = "topic_0"
consumer.subscribe([topic])

# Poll for new messages from Kafka and print them.
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            # Initial message consumption may take up to
            # `session.timeout.ms` for the consumer group to
            # rebalance and start consuming
            #print("Waiting...")
            pass
        elif msg.error():
            print("ERROR: %s".format(msg.error()))
        else:
            # Extract the (optional) key and value, and print.
            print("Consumed event from topic {topic}: key = {key:12} value = {value:12}".format(
                topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))
except KeyboardInterrupt:
    pass
finally:
    # Leave group and commit final offsets
    consumer.close()